<a href="https://colab.research.google.com/github/harshita23sharma/opensource_llms/blob/main/QNA_on_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.1.2
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
# !pip -q install datasets
# loralib
# sentencepiece
!pip -q install pypdf

!pip -q install sentence_transformers

In [ ]:
!pip install chromadb
!pip install tiktoken


In [4]:
from langchain.document_loaders import PyPDFLoader #load pdf and extract data
from langchain.document_loaders import TextLoader #upload csv and extract data from it
from langchain.document_loaders import Docx2txtLoader #upload docx and extract data from it

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login


import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import os
import sys


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Step 3: Load the Documents and Extract Text From Them


In [8]:
docs = "/content/drive/MyDrive/projects/qna_llama2"

In [23]:
from langchain.document_loaders import Docx2txtLoader


In [25]:
document=[]
for file in os.listdir(docs):
  if file.endswith(".pdf"):
    pdf_path=docs+ "/" + file
    loader=PyPDFLoader(pdf_path)
    document.extend(loader.load())
  # elif file.endswith('.docx') or file.endswith('.doc'):
  #   doc_path=docs+ "/" +file
  #   loader=Docx2txtLoader(doc_path)
  #   document.extend(loader.load())
  # elif file.endswith('.txt'):
  #   text_path=docs + "/" +file
  #   loader=TextLoader(text_path)
  #   document.extend(loader.load())

### Step 4: Split the Document into Chunks


In [26]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=100)


In [27]:
document_chunks=document_splitter.split_documents(document)

Step 5: Download the Embeddings from Hugging Face, Download the Sentence Transformer Embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


Step 6: Setting Up Chroma as our Vector Database


In [29]:
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')


In [30]:
vectordb.persist()


Step 7: Login into Hugging Face Account to Download the Model


In [ ]:
notebook_login()


Step 8: Download the Llama 2 7B Chat Model


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              #load_in_8bit=True,
                                              load_in_4bit=True
                                             )


Step 9: Creating a Hugging Face Pipeline


In [34]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30

              )

In [35]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


Step 10: Creating a memory object which is necessary to track inputs/outputs and hold a conversation

In [41]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [42]:
#Create our Q/A Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=vectordb.as_retriever(search_kwargs={'k':6}),
                                             verbose=False, memory=memory)


In [43]:
result=pdf_qa({"question":"llama2 is trained on which dataset"})


In [44]:
result

{'question': 'llama2 is trained on which dataset',
 'chat_history': [HumanMessage(content='llama2 is trained on which dataset'),
  AIMessage(content=' Based on the text, Llama 2 is trained on a new mix of data from publicly available sources.\n\nNote: The text does not explicitly state the name of the dataset used for pretraining, but it does mention that the data is from publicly available sources. Therefore, the answer is based on the information provided in the text and the context of the passage.')],
 'answer': ' Based on the text, Llama 2 is trained on a new mix of data from publicly available sources.\n\nNote: The text does not explicitly state the name of the dataset used for pretraining, but it does mention that the data is from publicly available sources. Therefore, the answer is based on the information provided in the text and the context of the passage.'}